<a href="https://colab.research.google.com/github/YuvanShankar2006/Production-Efficiency-and-Cost-Optimization/blob/main/Production_Efficiency_and_Cost_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Production Efficiency and Cost Optimization Project

## Project Overview

This project aims to develop an intelligent decision-support system for manufacturing production planning. The system will provide actionable insights on **which products should be produced and in what quantities** to maximize profitability, considering constraints and priorities related to production time and resource usage.

---

## Objectives

- Analyze historical production data to understand product-level time, cost, and resource consumption patterns.
- Build predictive models to estimate:
  - Production cycle time per unit
  - Resource usage per product (material, labor, energy)
  - Profitability metrics per product
- Incorporate user-defined priority sliders for:
  - **Time Priority**: Prioritize products with lower production time.
  - **Resource Priority**: Prioritize products with lower resource consumption.
- Develop an optimization framework to recommend product mix/production quantities that maximize profit under the given constraints and priorities.
- Provide insights to improve production efficiency and cost-effectiveness.

---

## Methodology

1. **Data Preparation**  
   Collect and preprocess production data including product details, production times, resource usage, costs, and outputs.

2. **Feature Engineering**  
   Extract relevant features like cycle times, tool usage durations, material consumption, labor hours, and downtime reasons.

3. **Model Development**  
   - Train regression models to predict production time, resource use, and profit per product unit.  
   - Define an optimization problem that incorporates predicted metrics and priority sliders.

4. **Optimization and Insights**  
   - Use linear programming to solve for optimal production quantities maximizing profit while respecting time and resource constraints.  
   - Allow tuning of production priorities via sliders affecting the optimization objective.

5. **Evaluation and Validation**  
   - Evaluate model accuracy and optimization results on historical data.  
   - Validate recommended production plans with business constraints.

---

## Tools & Technologies

- Python, Pandas, NumPy for data processing  
- Scikit-learn or similar for regression modeling  
- Optimization libraries: SciPy (linprog), PuLP, or CVXPY for linear programming  
- Jupyter / Google Colab as development environment  

---

## Expected Outcomes

- A flexible and interpretable model for production planning under multiple constraints.  
- Ability to simulate different priority scenarios (time vs resource) and see recommended production mixes.  
- Actionable insights for operations teams to improve efficiency and profitability.

---

*Let's start with data exploration and preparation in the next step.*


In [64]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np


In [65]:
df=pd.read_csv("/content/sample_production_data_500.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   work_order_id                   500 non-null    object 
 1   product_id                      500 non-null    object 
 2   product_name                    500 non-null    object 
 3   planned_quantity                500 non-null    int64  
 4   actual_quantity                 500 non-null    int64  
 5   good_units                      500 non-null    int64  
 6   defective_units                 500 non-null    int64  
 7   rework_units                    500 non-null    int64  
 8   start_time                      500 non-null    object 
 9   end_time                        500 non-null    object 
 10  actual_cycle_time_per_unit_min  500 non-null    float64
 11  downtime_minutes                500 non-null    int64  
 12  downtime_reason_code            500 

In [66]:
print(df.describe().T)
print(df.shape)


                                count        mean           std      min  \
planned_quantity                500.0   124.19600  4.440790e+01   50.000   
actual_quantity                 500.0   119.65800  4.459905e+01   42.000   
good_units                      500.0   116.66600  4.466790e+01   37.000   
defective_units                 500.0     1.97400  1.391833e+00    0.000   
rework_units                    500.0     1.01800  8.142494e-01    0.000   
actual_cycle_time_per_unit_min  500.0     1.40643  3.391212e-01    0.802   
downtime_minutes                500.0    29.47800  1.728720e+01    0.000   
number_of_tools_used            500.0     1.97600  8.029405e-01    1.000   
tool_usage_duration_1_min       500.0   102.60222  8.095900e+01    0.240   
tool_usage_duration_2_min       500.0    44.00748  5.883262e+01    0.000   
tool_usage_duration_3_min       500.0    17.67732  4.209177e+01    0.000   
material_qty_used               500.0   145.62756  5.890719e+01   43.370   
material_cos

In [67]:
identifiers = [
    'work_order_id', 'product_id', 'machine_id', 'operator_id'
]

product_info = [
    'product_name', 'planned_quantity', 'actual_quantity', 'good_units', 'defective_units', 'rework_units'
]

time_features = [
    'start_time', 'end_time', 'actual_cycle_time_per_unit_min', 'downtime_minutes',
    'tool_usage_duration_1_min', 'tool_usage_duration_2_min', 'tool_usage_duration_3_min'
]

resource_features = [
    'number_of_tools_used', 'material_qty_used', 'labor_hours', 'energy_kwh'
]

cost_features = [
    'material_cost_per_unit', 'labor_cost_per_hour', 'energy_cost_per_kwh', 'overhead_cost_per_hour'
]

calculated_outputs = [
    'unit_selling_price', 'profit_per_unit', 'actual_total_cost', 'cost_per_unit'
]

categorical_context = [
    'shift_code', 'downtime_reason_code'
]

print(" Identifiers:\n", identifiers)
print("\n Product Info:\n", product_info)
print("\n Time Features:\n", time_features)
print("\n Resource Features:\n", resource_features)
print("\n Cost Features:\n", cost_features)
print("\n Calculated Outputs:\n", calculated_outputs)
print("\n Context Info:\n", categorical_context)


 Identifiers:
 ['work_order_id', 'product_id', 'machine_id', 'operator_id']

 Product Info:
 ['product_name', 'planned_quantity', 'actual_quantity', 'good_units', 'defective_units', 'rework_units']

 Time Features:
 ['start_time', 'end_time', 'actual_cycle_time_per_unit_min', 'downtime_minutes', 'tool_usage_duration_1_min', 'tool_usage_duration_2_min', 'tool_usage_duration_3_min']

 Resource Features:
 ['number_of_tools_used', 'material_qty_used', 'labor_hours', 'energy_kwh']

 Cost Features:
 ['material_cost_per_unit', 'labor_cost_per_hour', 'energy_cost_per_kwh', 'overhead_cost_per_hour']

 Calculated Outputs:
 ['unit_selling_price', 'profit_per_unit', 'actual_total_cost', 'cost_per_unit']

 Context Info:
 ['shift_code', 'downtime_reason_code']


In [68]:
product_stats = df.groupby('product_name').agg({
    'profit_per_unit': 'mean',
    'actual_cycle_time_per_unit_min': 'mean',
    'material_qty_used': 'mean',
    'material_cost_per_unit': 'mean',
    'labor_hours': 'mean',
    'labor_cost_per_hour': 'mean',
    'energy_kwh': 'mean',
    'energy_cost_per_kwh': 'mean',
    'actual_quantity': 'mean'
}).reset_index()
product_stats['resource_cost'] = (
    product_stats['material_qty_used'] * product_stats['material_cost_per_unit'] +
    (product_stats['labor_hours'] * product_stats['labor_cost_per_hour']) / product_stats['actual_quantity'] +
    (product_stats['energy_kwh'] * product_stats['energy_cost_per_kwh']) / product_stats['actual_quantity']
)
product_stats = product_stats.rename(columns={
    'profit_per_unit': 'profit',
    'actual_cycle_time_per_unit_min': 'time',
    'resource_cost': 'resource'
})
product_stats[['product_name', 'profit', 'time', 'resource']]


,product_name,profit,time,resource
0,Product A,86.621856,1.430988,1443.636566
1,Product B,137.348065,1.362052,1390.962506
2,Product C,106.520112,1.422034,1459.046428


In [69]:
import ipywidgets as widgets
from IPython.display import display
time_slider = widgets.FloatSlider(
    value=1.0, min=0.0, max=3.0, step=0.1, description='Time Weight (w_t):'
)

resource_slider = widgets.FloatSlider(
    value=1.0, min=0.0, max=3.0, step=0.1, description='Resource Weight (w_r):'
)

display(time_slider, resource_slider)

FloatSlider(value=1.0, description='Time Weight (w_t):', max=3.0)

FloatSlider(value=1.0, description='Resource Weight (w_r):', max=3.0)

In [70]:
def compute_product_scores(w_t, w_r):
    df = product_stats.copy()
    df['profit_norm'] = (df['profit'] - df['profit'].min()) / (df['profit'].max() - df['profit'].min())
    df['time_norm'] = (df['time'] - df['time'].min()) / (df['time'].max() - df['time'].min())
    df['resource_norm'] = (df['resource'] - df['resource'].min()) / (df['resource'].max() - df['resource'].min())
    df['score'] = df['profit_norm'] - w_t * df['time_norm'] - w_r * df['resource_norm']

    return df[['product_name', 'profit', 'time', 'resource', 'score']].sort_values(by='score', ascending=False)


In [74]:
from scipy.optimize import linprog
T_max = 80000
R_max = 10

def optimize_production(w_t, w_r, T_max, R_max):
    data = compute_product_scores(w_t, w_r)
    c = -data['score'].values

    A = [
        data['time'].values,
        data['resource'].values
    ]
    b = [T_max, R_max]
    bounds = [(0, None)] * len(data)
    result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='highs')

    if result.success:
        quantities = result.x
        output = data.copy()
        output['recommended_qty'] = np.round(quantities).astype(int)
        return output[['product_name', 'score', 'recommended_qty']]
    else:
        return "Optimization failed: " + result.message
optimize_production(time_slider.value, resource_slider.value, T_max, R_max)


,product_name,score,recommended_qty
1,Product B,1.000000,0
2,Product C,-5.218055,0
0,Product A,-5.320991,0


In [75]:
def interactive_optimizer(w_t, w_r):
    clear_output(wait=True)  # Clears previous output so it refreshes nicely

    # Compute scores with normalization (use the fixed function I shared)
    df_scores = compute_product_scores(w_t, w_r)

    # Run optimization with current weights and generous constraints
    T_max = 800000
    R_max = 100000
    optimized = optimize_production(w_t, w_r, T_max, R_max)

    # Display results
    display(df_scores.style.format({'score': '{:.3f}'}))
    print("\nRecommended production quantities:")
    display(optimized)


In [76]:
import ipywidgets as widgets
widgets.interact(interactive_optimizer, w_t=time_slider, w_r=resource_slider);


interactive(children=(FloatSlider(value=3.0, description='Time Weight (w_t):', max=3.0), FloatSlider(value=3.0…